In [0]:
%run ./common_functions

In [0]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim,regexp_replace,current_date
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql import SparkSession
spark = SparkSession.getActiveSession()

table_name="dataValidation_Output_table" 
path = eh_folder_path+"/"+table_name

In [0]:
#df = spark.read.format("delta").table("`cross_ds-unitycatalog-dev`.`raw-ds-gsap`.dataValidation_Control_table")

"""
Data validation control table
return: Dataframe
"""
df_DV = spark.read.table(f"`{uc_catalog_name}`.`{uc_raw_schema}`.dataValidation_Control_table")
spark.sql(f"Delete from `{uc_catalog_name}`.`{uc_eh_schema}`.dataValidation_Output_table")

for row in df_DV.collect():
    table_name = row["Table_name"]
    measure = row["Measure"]
    date_field = row["Date_field"]
    From_Ind = row["From_Ind"]
    To_Ind = row["To_Ind"]

    query = f"""
            select 
                '{table_name}' AS Table_name,
                count(*) AS Record_Count,
                '{measure}' AS Measure,
                sum({measure}) AS Measure_Sum,
                '{From_Ind}',
                '{To_Ind}'
            from 
                 `{uc_catalog_name}`.`{uc_euh_schema}`.{table_name}
            where
                {date_field} > current_date() - {From_Ind} AND {date_field} < current_date() - {To_Ind}
    """

    df_output = spark.sql(query);
    df_output.withColumn('Measure',regexp_replace('Measure',"`","")).withColumn('ingested_at',current_date()).write.mode('append').insertInto(f"`{uc_catalog_name}`.`{uc_eh_schema}`.dataValidation_Output_table")
